# 🔒 SCA Package Model Training - Google Colab (GitHub Dataset)

Train CodeLlama 7B to detect package vulnerabilities using 2024-2025 CVE data

**Before starting:**
1. Enable GPU: Runtime → Change runtime type → T4 GPU
2. Dataset will download automatically from GitHub
3. Run cells one by one with Shift+Enter

**Dataset:** 2024-2025 CVEs (~10,000 training examples, ~200MB)

## Step 1: Check GPU

In [ ]:
import torch
print(f"🖥️  GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"📊 GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("❌ No GPU! Go to Runtime → Change runtime type → Select T4 GPU")

🖥️  GPU Available: False
❌ No GPU! Go to Runtime → Change runtime type → Select T4 GPU


## Step 2: Install Dependencies

In [ ]:
%%capture
# Silent installation (remove %%capture to see output)
!pip install -q transformers==4.37.0 datasets==2.16.0 peft==0.8.0 accelerate==0.26.0 sentencepiece

# Try to install bitsandbytes - if it fails, we'll skip quantization
try:
    !pip install -q bitsandbytes==0.43.0  # Newer version with better CUDA support
    print("✅ Dependencies installed (with bitsandbytes)")
except:
    print("⚠️  bitsandbytes installation failed - will skip quantization")
    print("✅ Dependencies installed (without bitsandbytes)")

## Step 3: Download Dataset from GitHub

In [ ]:
import os

# GitHub repository details
GITHUB_REPO = "abhay2510kr/ai_sec"
DATASET_FILE = "datasets/sca_training_2024_2025.json"
DATASET_URL = f"https://raw.githubusercontent.com/{GITHUB_REPO}/main/{DATASET_FILE}"

print(f"📥 Downloading dataset from GitHub...")
print(f"Repository: {GITHUB_REPO}")
print(f"File: {DATASET_FILE}")

# Download dataset
!wget -q --show-progress {DATASET_URL} -O /content/sca_training_dataset.json

# Verify download
if os.path.exists('/content/sca_training_dataset.json'):
    file_size = os.path.getsize('/content/sca_training_dataset.json') / (1024 * 1024)
    print(f"\n✅ Dataset downloaded successfully!")
    print(f"📊 File size: {file_size:.2f} MB")
else:
    print("\n❌ Download failed! Check if dataset exists in GitHub repo")
    print(f"URL: {DATASET_URL}")

📥 Downloading dataset from GitHub...
Repository: abhay2510kr/ai_sec
File: datasets/sca_training_2024_2025.json
/content/sca_traini 100%[===================>]  14.21M  --.-KB/s    in 0.03s   

✅ Dataset downloaded successfully!
📊 File size: 14.21 MB


## Step 4: Load and Prepare Dataset

In [ ]:
from datasets import load_dataset

DATASET_PATH = "/content/sca_training_dataset.json"

print(f"📂 Loading dataset from: {DATASET_PATH}")

# Load dataset
dataset = load_dataset('json', data_files=DATASET_PATH)

# Split into train/validation
dataset = dataset['train'].train_test_split(test_size=0.1, seed=42)

print(f"\n📊 Dataset Statistics:")
print(f"  Training samples: {len(dataset['train'])}")
print(f"  Validation samples: {len(dataset['test'])}")

# Show sample
print(f"\n📝 Sample training example:")
print(dataset['train'][0]['text'][:600] + "...")

## Step 5A: Clear GPU Memory (Run this first if reloading model)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

print("📥 Loading CodeLlama-7b-Instruct...")
print("  Loading in float16 (no quantization)")
print("  This avoids bitsandbytes compatibility issues")

if torch.cuda.is_available():
    model = AutoModelForCausalLM.from_pretrained(
        "codellama/CodeLlama-7b-Instruct-hf",
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
    )
    print("  ✅ Model loaded in float16 (~14 GB)")
else:
    model = AutoModelForCausalLM.from_pretrained(
        "codellama/CodeLlama-7b-Instruct-hf",
        torch_dtype=torch.float32,
        low_cpu_mem_usage=True,
        trust_remote_code=True,
    )
    print("  ✅ Model loaded on CPU")

tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-Instruct-hf")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("\n✅ Model and tokenizer ready!")

## Step 5B: Load Model (Float16 - No Quantization)

In [ ]:
# Clear GPU memory if model already loaded
import gc
import torch

try:
    del model
    del tokenizer
    print("🗑️  Cleared previous model from memory")
except:
    print("ℹ️  No previous model to clear")

gc.collect()
torch.cuda.empty_cache()

print("✅ GPU memory cleared")
print("💡 Now run Step 5B to load the model")

## Step 6: Configure LoRA (Train only 0.5% of parameters)

In [ ]:
from peft import LoraConfig, get_peft_model

# Check if model was loaded in Step 5
try:
    model
except NameError:
    print("❌ ERROR: Model not loaded!")
    print("📌 Please run Step 5 first to load the model")
    raise NameError("Model not found. Run Step 5 (Load Model) before this cell.")

# LoRA configuration - this trains only 0.5% of parameters!
lora_config = LoraConfig(
    r=16,  # Rank (lower = less parameters, faster training)
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

print("ℹ️  Applying LoRA (training only 0.5% of model parameters)")
model = get_peft_model(model, lora_config)

# Show trainable parameters
model.print_trainable_parameters()
# Expected: trainable params: ~40M / 7B (~0.5%!)

## Step 7: Tokenize Dataset

In [ ]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=2048,
        padding="max_length",
    )

print("🔄 Tokenizing dataset...")

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

print("✅ Dataset tokenized!")

## Step 8: Configure Training

In [ ]:
from transformers import TrainingArguments, Trainer

# Check if GPU is available
use_fp16 = torch.cuda.is_available()

training_args = TrainingArguments(
    output_dir="/content/sca-package-checkpoints",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,  # Effective batch size = 16
    learning_rate=2e-4,
    fp16=use_fp16,  # Only use fp16 if GPU available
    save_strategy="steps",
    save_steps=100,
    logging_steps=10,
    warmup_steps=50,
    optim="adamw_torch",  # Use standard optimizer (no 8bit)
    gradient_checkpointing=True,
    max_grad_norm=0.3,
    report_to="none",
    eval_strategy="steps",
    eval_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)

print("✅ Trainer configured!")
if use_fp16:
    print(f"\n⏱️  Estimated training time: 4-6 hours on T4 GPU")
else:
    print(f"\n⏱️  Estimated training time: 24+ hours on CPU")
print(f"💾 Checkpoints will be saved to: /content/sca-package-checkpoints")
print(f"⚠️  Remember to download the model before session ends!")

## Step 9: START TRAINING! 🚀

In [ ]:
print("🚀 Starting training...")
print("⏰ This will take 2-4 hours")
print("💡 TIP: You can close this tab - training will continue!")
print("\n" + "="*60)

trainer.train()

print("\n" + "="*60)
print("✅ Training complete!")
print("="*60)

## Step 10: Save Final Model

In [ ]:
output_dir = "/content/sca-package-final"

print(f"💾 Saving final model to: {output_dir}")

trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print("✅ Model saved successfully!")
print(f"\n📁 Model location: {output_dir}")
print("\n⬇️  IMPORTANT: Download the model now!")
print("   Run the next cell to create a downloadable zip file")

In [ ]:
import shutil
from google.colab import files

# Create zip file
print("📦 Creating zip file...")
shutil.make_archive('/content/sca-package-final', 'zip', '/content/sca-package-final')

print("✅ Zip file created!")
print(f"📊 Size: {os.path.getsize('/content/sca-package-final.zip') / (1024*1024):.2f} MB")

# Download
print("\n⬇️  Starting download...")
files.download('/content/sca-package-final.zip')

print("\n✅ Download complete!")
print("💡 Save this file - you can use it to run inference later!")

## Step 11: Download Model (IMPORTANT!)

## Step 12: Test the Model! 🧪

In [ ]:
print("🧪 Testing the trained model...\n")

# Test input
test_input = """[INST] Analyze this package.json for known vulnerabilities

```json
{
  "name": "my-app",
  "dependencies": {
    "express": "4.16.0",
    "lodash": "4.17.4",
    "axios": "0.18.0"
  }
}
``` [/INST]"""

# Tokenize and generate
inputs = tokenizer(test_input, return_tensors="pt").to("cuda")
outputs = model.generate(
    **inputs,
    max_new_tokens=512,
    temperature=0.1,
    do_sample=True,
    top_p=0.95
)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("="*60)
print("🤖 MODEL OUTPUT:")
print("="*60)
print(result)
print("="*60)

## 🎉 Congratulations!

You've successfully trained your SCA model on 2024-2025 CVE data!

**What you have:**
- ✅ Trained model (downloaded as zip)
- ✅ Can detect package vulnerabilities
- ✅ Ready for production use

**Next Steps:**
1. Train more models: SAST, IaC, Container, etc.
2. Deploy the model using vLLM or Ollama
3. Integrate into your CI/CD pipeline

**Model Info:**
- Base: CodeLlama-7b-Instruct
- Training data: 2024-2025 CVEs (~10K examples)
- Training time: ~4-6 hours
- Model size: ~7GB (quantized)